In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt

In [170]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Conv2D, Concatenate, MaxPool2D, Reshape, Flatten, Dropout
from keras.layers import ReLU
from keras.layers import BatchNormalization, LSTM, Bidirectional

from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.layers import Add

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
from keras import backend as K

In [4]:
# !python3 -m pip install gensim

In [5]:
from gensim.models import Word2Vec

In [6]:
%matplotlib inline

In [7]:
# !python3 -m pip install -U nltk
# !python3 -m nltk.downloader punkt

In [8]:
import nltk

In [9]:
from nltk.corpus import stopwords

In [17]:
path = 'epi_r.csv'
# path = 'recipes_with_calories.csv'

In [23]:
df = pd.read_csv(path)
# df = pd.read_csv(path, sep="|", index_col=0)

In [24]:
all_columns = list(df.columns)
len(all_columns)

680

In [168]:
df.dropna(axis=0).shape, df.shape

((15864, 680), (20052, 680))

In [169]:
for column in df.columns:
    if df[column].isnull().any():
        print(column)
#         df[column] = df[column].fillna(df[column].mean())

protein
fat
sodium


In [28]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000000,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000000,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.000000,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,252.445801,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.000000,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def get_small_dict(df, keep_columns, index):
    my_dict = df.iloc[index][keep_columns].to_dict()
    my_dict['tags'] = [
        key for key, value in df.iloc[index].to_dict().items() if value != 0 and key not in keep_columns]
    my_dict['title'] = my_dict['title'].strip()
    my_dict['tags'] = [
        str(unicodedata.normalize('NFKD', t).encode('ascii','ignore'), encoding='ascii') for t in my_dict['tags']
    ]
    return my_dict

In [32]:
keep_columns = ['title', 'rating', 'calories', 'protein', 'fat','sodium']

In [171]:
def remove_outliers_with_tukey(df):
    print('There were {} samples before outlier removal.'.format(len(df)))

    # Remove calories outliers by the Tukey rule
    Q3 = np.percentile(df[regr_column], 75)
    Q1 = np.percentile(df[regr_column], 25)
    IQR = Q3 - Q1

    outliers = df[regr_column].loc[
        (df[regr_column] < (Q1 - 1.5 * IQR)) | 
        (df[regr_column] > (Q3 + 1.5 * IQR))
    ].index
    
    df = df.drop(outliers)
    
    print('There are now {} samples.'.format(len(cut_df)))

    return df

In [ ]:
# pick regression column
regr_column = 'calories'
cut_df = df[df[regr_column] == df[regr_column]]

cut_df = remove_outliers_with_tukey(df=cut_df)

# transform df in list of dictionaries for processing
regr_data = []
for i in range(len(cut_df)):
    regr_data.append(get_small_dict(cut_df, keep_columns, i))

In [176]:
def remove_punctuation_signs(regr_data):
    for d in regr_data:
        if 'tags' not in d:
            raise ValueError('Expected to have <tags> key in each of the regression input data dictionary!')
    corpus = ['|'.join(
        d['tags'] + 
        [w.lower() for w in nltk.word_tokenize(d['title']) if
             w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
    ) for d in regr_data]
    return corpus

In [177]:
# make sure the data doesn't have any punctuation signs
corpus = remove_punctuation_signs(regr_data)

In [178]:
# pick tokenization method
def tokenize(word_list):
    return word_list.split('|')

In [179]:
def get_y_labels(regr_column, regr_data):
    return [d[regr_column] for d in regr_data]

In [180]:
y_label = get_y_labels(regr_column, regr_data)

In [183]:
if len(corpus) != len(y_label):
    print('[ERROR] There is a mismatch between the size of the input data ' +
          ' ({} elements) and the size of labels ({} elements)!'.format(len(corpus), len(y_label)))

## Try a vanilla regression (CountVectorizer + LinearRegression from sklearn)

In [191]:
# fit vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize)
X = vectorizer.fit_transform(corpus)

# train, test data
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y_label, test_size=0.2, random_state=42)

# sklearn LinearRegression model
regr_sklearn = LinearRegression()
regr_sklearn.fit(X_train, y_train)

# predict data
y_pred = regr_sklearn.predict(X_test)

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", mean_squared_error(y_true=y_test, y_pred=y_pred))

# show first k differences
first_k = 10
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  1.2084885248453593e+27
137.68478486181857 919.0
86.36782347963987 116.0
283.02489564062716 228.0
539.5003595078147 846.029052734375
168.97008607031466 145.0
690.1912774765647 466.0
589.8633966171897 193.0
168.33516967104612 699.3670654296875
324099096238.38324 116.0
224.83263489843966 194.05316162109372


## Try Word2Vec embeddings + Keras NN

### Word2Vec embedding

In [192]:
vocab = set()
for word_list in corpus:
    vocab |= set(tokenize(word_list))

print("Choose embedding size between {} and {}".format(np.log2(len(vocab)), np.power(len(vocab), 1/4) * 10))

w2v_corpus = []
for word_list in corpus:
    w2v_corpus.append(tokenize(word_list))

Choose embedding size between 12.742519971661299 and 90.9850803511905


In [193]:
EMBEDDING_SIZE = 50

In [194]:
w2v_model = Word2Vec(w2v_corpus, size=EMBEDDING_SIZE, window=5, min_count=1, workers=-1)

print("Embedding size is {}".format(w2v_model.wv.vector_size))

Embedding size is 50


In [195]:
sequence_lengths = [len(w) for w in w2v_corpus]

print("Sequence length should be {} ".format(pd.Series(sequence_lengths).describe()['mean']))

Sequence length should be 17.08845909955612 


In [196]:
FIXED_LENGTH = 17

In [197]:
def generate_embeddings_matrix(sample, model, fixed_length=17):
    embedding_size = model.wv.vector_size
    embeddings = []
    for sen in sample:
        for word in sen:
            if word not in model.wv.vocab:
                continue
            embeddings.append(model.wv[word])
            if len(embeddings) >= fixed_length:
                break

    embeddings = np.array(embeddings)

    while len(embeddings) < fixed_length:
        padding_length = fixed_length - len(embeddings)
        embeddings = np.concatenate((embeddings, embeddings[:padding_length]))
    if len(embeddings) > fixed_length:
        embeddings = embeddings[:fixed_length]

    return embeddings

In [198]:
def generate_input_matrix(sample, fixed_length=17):
    
    inputs = np.array(sample)

    while len(inputs) < fixed_length:
        padding_length = fixed_length - len(inputs)
        inputs = np.concatenate((inputs, inputs[:padding_length]))
    if len(inputs) > fixed_length:
        inputs = inputs[:fixed_length]

    return inputs

In [200]:
# Generate list of input data (embedding matrix) and make sure every input has FIXED_LENGTH elements
X = []
for sample in w2v_corpus:
    embedded_sample = generate_embeddings_matrix(sample, w2v_model, FIXED_LENGTH)
    if embedded_sample.shape != (FIXED_LENGTH, EMBEDDING_SIZE):
        print(embedded_sample.shape)
    X.append(embedded_sample)

In [201]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

### Keras NN

In [202]:
FIXED_LENGTH = 17
EMBEDDING_SIZE = 50

BATCH_SIZE = 16
EPOCHS=200

In [203]:
def baseline_model():
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH, EMBEDDING_SIZE), name='input')
    flat = Flatten()(input_layer)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(flat)
    output = Dense(1, kernel_initializer='normal')(z)
    model = Model(inputs=[input_layer], outputs=[output])
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [204]:
model = baseline_model()

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

In [206]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [207]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/200
15139/15139 [==============================] - 7s 461us/step - loss: 177492.8872
Epoch 2/200
15139/15139 [==============================] - 3s 181us/step - loss: 101400.8843
Epoch 3/200
15139/15139 [==============================] - 3s 168us/step - loss: 62622.7186
Epoch 4/200
15139/15139 [==============================] - 3s 185us/step - loss: 58213.8089
Epoch 5/200
15139/15139 [==============================] - 3s 190us/step - loss: 57806.6550
Epoch 6/200
15139/15139 [==============================] - 3s 221us/step - loss: 57532.3931
Epoch 7/200
15139/15139 [==============================] - 4s 236us/step - loss: 57284.5530
Epoch 8/200
15139/15139 [==============================] - 3s 195us/step - loss: 57067.9269
Epoch 9/200
15139/15139 [==============================] - 4s 237us/step - loss: 56871.3663
Epoch 10/200
15139/15139 [==============================] - 3s 227us/step - loss: 56702.4237
Epoch 11/200
15139/15139 [==============================] - 3s 176us/step - l

15139/15139 [==============================] - 2s 140us/step - loss: 53907.4016
Epoch 177/200
15139/15139 [==============================] - 2s 139us/step - loss: 53904.4644
Epoch 178/200
15139/15139 [==============================] - 2s 143us/step - loss: 53907.8569
Epoch 179/200
15139/15139 [==============================] - 2s 138us/step - loss: 53910.0248
Epoch 180/200
15139/15139 [==============================] - 2s 162us/step - loss: 53899.1603
Epoch 181/200
15139/15139 [==============================] - 2s 150us/step - loss: 53906.3608
Epoch 182/200
15139/15139 [==============================] - 2s 136us/step - loss: 53902.2458
Epoch 183/200
15139/15139 [==============================] - 2s 134us/step - loss: 53904.5250
Epoch 184/200
15139/15139 [==============================] - 2s 135us/step - loss: 53898.7430
Epoch 185/200
15139/15139 [==============================] - 2s 138us/step - loss: 53902.6838
Epoch 186/200
15139/15139 [==============================] - 2s 139us/step

In [210]:
y_pred = model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", mean_squared_error(y_true=y_test, y_pred=y_pred))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  54058.00614624972
Showing the first 10 differences between predicted and real.
353.14505 919.0
327.23587 116.0
341.66547 228.0
402.91074 846.029052734375
356.3785 145.0
330.4774 466.0
400.0054 193.0
292.58456 699.3670654296875
312.0872 116.0
314.25296 194.05316162109372


## Keras embedding + NN

### Generate input data 

In [214]:
X = []
for sample in w2v_corpus:
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Encode the words

In [215]:
encoder = LabelEncoder()
encoder.fit(np.array(X).flatten())

LabelEncoder()

In [216]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

### Split X in train and test

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_label, test_size=0.2, random_state=42)

In [218]:
INPUT_SHAPE = (FIXED_LENGTH, )
INPUT_SHAPE

(17,)

### Define the Keras embedded model 

In [219]:
def keras_embedding_model(encoder):
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH,), name='input')
    embedded_text = Embedding(input_dim=len(encoder.classes_),
                              output_dim=EMBEDDING_SIZE,
                              name='text_embedding')(input_layer)
    embedded_text = Flatten()(embedded_text)
    z = BatchNormalization()(embedded_text)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.7)(z)
    z = ReLU()(z)
    output = Dense(1, kernel_initializer='normal', activation='linear')(z)
    model = Model(inputs=[input_layer], outputs=[output])

    # Compile model
    model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')

    return model

In [220]:
keras_embedded_model = keras_embedding_model(encoder=encoder)

In [221]:
keras_embedded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 17)                0         
_________________________________________________________________
text_embedding (Embedding)   (None, 17, 50)            280650    
_________________________________________________________________
flatten_18 (Flatten)         (None, 850)               0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 850)               3400      
_________________________________________________________________
fc1 (Dense)                  (None, 32)                27232     
_________________________________________________________________
batch_normalization_29 (Batc (None, 32)                128       
_________________________________________________________________
dropout_24 (Dropout)         (None, 32)                0         
__________

In [222]:
def schedule(epoch, lr):
    if epoch in [500]:
        return lr / 3.0
    return lr
scheduler = LearningRateScheduler(schedule=schedule)

In [226]:
y_series = pd.Series(y_label)
std_y, mean_y = y_series.describe()['std'], y_series.describe()['mean']

def rmse(y_true, y_pred):
    y_true = y_true * std_y + mean_y
    y_pred = y_pred * std_y + mean_y
    return K.sqrt(K.mean(K.square(y_true - y_pred), axis=0))

In [227]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [228]:
X_train.shape, y_train.shape

((15139, 17), (15139,))

In [229]:
history = keras_embedded_model.fit(X_train, y_train, epochs=EPOCHS,
                    callbacks=[scheduler, EarlyStopping(patience=10, monitor='rmse'),
                               ModelCheckpoint('models/embedding.h5', monitor='rmse')],
                    batch_size=BATCH_SIZE)

Epoch 1/200
15139/15139 [==============================] - 11s 751us/step - loss: 73732.6405


/home/skadi/.local/lib/python3.5/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `rmse` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
15139/15139 [==============================] - 8s 505us/step - loss: 42618.8289
Epoch 3/200
15139/15139 [==============================] - 10s 639us/step - loss: 38337.6373
Epoch 4/200
15139/15139 [==============================] - 10s 633us/step - loss: 35543.6541
Epoch 5/200
15139/15139 [==============================] - 8s 542us/step - loss: 34883.6313
Epoch 6/200
15139/15139 [==============================] - 8s 552us/step - loss: 33436.5811
Epoch 7/200
15139/15139 [==============================] - 8s 560us/step - loss: 32306.8438
Epoch 8/200
15139/15139 [==============================] - 9s 598us/step - loss: 30196.2253
Epoch 9/200
15139/15139 [==============================] - 8s 549us/step - loss: 30786.4307
Epoch 10/200
15139/15139 [==============================] - 9s 576us/step - loss: 29025.4467
Epoch 11/200
15139/15139 [==============================] - 9s 562us/step - loss: 28705.7763
Epoch 12/200
15139/15139 [==============================] - 8s 546us/step - 

15139/15139 [==============================] - 7s 486us/step - loss: 17756.2605
Epoch 91/200
15139/15139 [==============================] - 7s 476us/step - loss: 18088.6039
Epoch 92/200
15139/15139 [==============================] - 7s 477us/step - loss: 17923.1772
Epoch 93/200
15139/15139 [==============================] - 7s 476us/step - loss: 18130.4486
Epoch 94/200
15139/15139 [==============================] - 7s 475us/step - loss: 17403.0934
Epoch 95/200
15139/15139 [==============================] - 7s 466us/step - loss: 17733.5411
Epoch 96/200
15139/15139 [==============================] - 7s 474us/step - loss: 18063.4087
Epoch 97/200
15139/15139 [==============================] - 7s 491us/step - loss: 17074.3758
Epoch 98/200
15139/15139 [==============================] - 7s 480us/step - loss: 17575.0455
Epoch 99/200
15139/15139 [==============================] - 8s 533us/step - loss: 17979.3879
Epoch 100/200
15139/15139 [==============================] - 7s 479us/step - loss: 

In [ ]:
y_pred = keras_embedded_model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", mean_squared_error(y_true=y_test, y_pred=y_pred))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

### Save model

In [ ]:
keras_embedded_model.save('models/predict_calories_model')

### Get the dataframe that has NaN values for our regression column (calories)

In [ ]:
predict_df = df[df[regr_column] != df[regr_column]]

### Get the input data we need to predict the needed values

In [ ]:
predict_data = []
for i in range(len(predict_df)):
    predict_data.append(get_small_dict(predict_df, keep_columns, i))
corpus = ['|'.join(
    d['tags'] + 
    [w.lower() for w in nltk.word_tokenize(d['title']) if w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
) for d in predict_data]

### Split input data using the tokenizer

In [ ]:
predict_corpus = []
for word_list in corpus:
    predict_corpus.append(tokenize(word_list))

### Make sure input data has a fixed shape

In [ ]:
X = []
for sample in predict_corpus:
    sample = [w for w in sample if w in encoder.classes_]
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Transform matrix-like input data using available encoder

In [ ]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

In [ ]:
X_encoded = np.array(X_encoded)
X_encoded.shape

### Predict missing data 

In [ ]:
y_pred = keras_embedded_model.predict(X_encoded)

y_pred = y_pred.flatten()

### Fill the dataframe with predicted values

In [ ]:
predict_df[regr_column] = y_pred

### Update initial dataframe with missing values

In [ ]:
update_regr_column = [np.nan for _ in range(len(df))]
for idx, value in zip(predict_df.index, predicted):
    update_regr_column[idx] = value

In [ ]:
df.update({regr_column: update_regr_column})

### Save updated dataframe

In [ ]:
df.to_csv('recipes_with_calories.csv', sep='|')